Using kernel `conda_pytorch_latest_p36`

In [1]:
import sys
sys.path.append('../../../')

In [2]:
from pathlib import Path
import os
import random
import json


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch
import boto3
from transformers import (
    AutoTokenizer, 
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments, 
    Trainer
)
import sagemaker
from sagemaker import get_execution_role
from sagemaker.transformer import Transformer
from sagemaker.pytorch import PyTorchModel

In [4]:
from deep.constants import *

In [5]:
%load_ext autoreload
%autoreload 2

## Sagemaker Prep

In [6]:
default_bucket = 'deep-experiments-sagemaker-bucket'
sess = sagemaker.Session(default_bucket=default_bucket)

role = 'AmazonSageMaker-ExecutionRole-20210519T102514'
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = SAGEMAKER_BUCKET
prefix = "huggingface/first"  # Replace with the prefix under which you want to store the data if needed


AmazonSageMaker-ExecutionRole-20210519T102514


### Bucket upload

In [7]:
pt_mnist_model_data = 's3://sagemaker-us-east-1-961104659532/pytorch-training-2021-05-26-13-34-05-285/output/model.tar.gz'

In [8]:
hyperparameters={
    'train_batch_size': 32,
    'model_name': 'distilbert-base-uncased'
}

model = PyTorchModel(
    entry_point="batch_inference.py",
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/pytorch_estimator_base'),
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.8.1",
    py_version="py3",
)

In [9]:
# then create transformer from PyTorchModel object
transformer = model.transformer(
    instance_count=1, 
    instance_type='ml.m5.large', 
    strategy='MultiRecord',
    assemble_with='Line',
    output_path='s3://{}/batch_transform_output'.format(bucket),
)

In [10]:
dummy_data = pd.DataFrame({"inputs": ['There are many health problems', 'There are many more health problems']})
dummy_data.to_csv('s3://deep-experiments-sagemaker-bucket/test1/prova.csv')

In [11]:
transformer.transform(
    's3://deep-experiments-sagemaker-bucket/test1/prova.csv',
    content_type='text/csv', 
    split_type='Line', 
    logs=True, 
)

.............................Collecting transformers==4.6.1
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=ad332aea1f5dc6704b5536c8d5a5a845462e5cfa5a24224606fadb6ae8fdba33
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built idna-ssl



2021-05-27 14:14:51,840 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.3.1
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 986 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model.mar
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metrics report format: prometheus
Enable metrics API: true
2021-05-27 14:14:51,893 [INFO ] main org.pytorch.serve.ModelServer - Loading initial models: model.mar
2021-05

2021-05-27T14:14:59.253:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
